In [7]:
import geopandas as gpd 

In [8]:
tiles = gpd.read_file("../data/tiles.geojson")
trees_data = gpd.read_file("../data/trees.geojson")

In [9]:
tiles

,id,title,geometry
0,"Tile(x=6773, y=293571, z=19)","XYZ tile Tile(x=6773, y=293571, z=19)","POLYGON ((-175.34935 -21.08706, -175.34935 -21..."
1,"Tile(x=6773, y=293572, z=19)","XYZ tile Tile(x=6773, y=293572, z=19)","POLYGON ((-175.34935 -21.0877, -175.34935 -21...."
2,"Tile(x=6773, y=293573, z=19)","XYZ tile Tile(x=6773, y=293573, z=19)","POLYGON ((-175.34935 -21.08834, -175.34935 -21..."
3,"Tile(x=6773, y=293574, z=19)","XYZ tile Tile(x=6773, y=293574, z=19)","POLYGON ((-175.34935 -21.08898, -175.34935 -21..."
4,"Tile(x=6773, y=293575, z=19)","XYZ tile Tile(x=6773, y=293575, z=19)","POLYGON ((-175.34935 -21.08963, -175.34935 -21..."
...,...,...,...
546,"Tile(x=6791, y=293595, z=19)","XYZ tile Tile(x=6791, y=293595, z=19)","POLYGON ((-175.33699 -21.10244, -175.33699 -21..."
547,"Tile(x=6791, y=293596, z=19)","XYZ tile Tile(x=6791, y=293596, z=19)","POLYGON ((-175.33699 -21.10308, -175.33699 -21..."
548,"Tile(x=6791, y=293597, z=19)","XYZ tile Tile(x=6791, y=293597, z=19)","POLYGON ((-175.33699 -21.10372, -175.33699 -21..."
549,"Tile(x=6791, y=293598, z=19)","XYZ tile Tile(x=6791, y=293598, z=19)","POLYGON ((-175.33699 -21.10436, -175.33699 -21..."


In [10]:
from geomltoolkits.utils import split_geojson_by_tiles

In [13]:
split_geojson_by_tiles(tiles, trees_data, output_dir="../data/labels")

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/pyogrio/raw.py:200: RuntimeWarning: Invalid range specified 
  return ogr_read(


DataSourceError: id                                  title  \0    Tile(x=6773, y=293571, z=19)  XYZ tile Tile(x=6773, y=293571, z=19)   1    Tile(x=6773, y=293572, z=19)  XYZ tile Tile(x=6773, y=293572, z=19)   2    Tile(x=6773, y=293573, z=19)  XYZ tile Tile(x=6773, y=293573, z=19)   3    Tile(x=6773, y=293574, z=19)  XYZ tile Tile(x=6773, y=293574, z=19)   4    Tile(x=6773, y=293575, z=19)  XYZ tile Tile(x=6773, y=293575, z=19)   ..                            ...                                    ...   546  Tile(x=6791, y=293595, z=19)  XYZ tile Tile(x=6791, y=293595, z=19)   547  Tile(x=6791, y=293596, z=19)  XYZ tile Tile(x=6791, y=293596, z=19)   548  Tile(x=6791, y=293597, z=19)  XYZ tile Tile(x=6791, y=293597, z=19)   549  Tile(x=6791, y=293598, z=19)  XYZ tile Tile(x=6791, y=293598, z=19)   550  Tile(x=6791, y=293599, z=19)  XYZ tile Tile(x=6791, y=293599, z=19)                                                 geometry  0    POLYGON ((-175.34935 -21.08706, -175.34935 -21...  1 